In [ ]:
import numpy as np
import os
import pandas as pd
import plotly.express as px
import zipfile
import glob
import cv2 
import matplotlib.pyplot as plt
import random
from tensorflow.keras.applications import  Xception,VGG16,InceptionResNetV2
from tensorflow.keras.layers import Conv2D,BatchNormalization,Dropout,Flatten,Input,Lambda,MaxPooling2D,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2  
from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
from keras.engine.base_layer import Layer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
seed=42
np.random.seed=seed

# Extractfiles

In [ ]:
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip')as z:
    z.extractall()
    z.close()
    

with zipfile.ZipFile('../input/dogs-vs-cats/train.zip') as z:
    z.extractall()
    z.close()

In [ ]:
image_path='/kaggle/working'

In [ ]:
# get label Name   
def get_Label(number):
    labels = {0:'Cat', 1:'Dog'}
    return labels[number]


#plot predction function
def plot_predection(model_name):
    plt.figure(figsize=(20,15))
    plt.suptitle("Predection  Images", fontsize=20)
    images = [] 
    path =image_path+'/'+'test1/'
    count = 0  #val_images,val_labels
    for i,files in enumerate(os.listdir(path)) :
        img = plt.imread(path+files)
        img = cv2.resize(img,(128,128))
        plt.imshow(img,cmap=plt.cm.binary)
        img = np.expand_dims(img, axis=0)
        feature = model_name.predict(img)
        predection  = np.argmax(feature, axis=1)
        plt.subplot(5,7,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel("Predicted"+get_Label(int(predection)))
        count += 1
        if count == 34 :
            break 

In [ ]:
all_categories=[]
images=os.listdir(image_path+'/'+'train')

for file in images:
    if file.split('.')[0]=='dog':
        all_categories.append(1)
        
    else:
        all_categories.append(0)

    
df=pd.DataFrame({'image' :images , 'category' : all_categories})
df.head()

In [ ]:
print(f'There are {len(images)} for Trainiing ')


In [ ]:
df.category.value_counts()

In [ ]:

fig=px.histogram(df,
             x='category',
             color='category',
             title='Train label Visuilization',pattern_shape='category')
fig.show()

In [ ]:
import seaborn as sns
plt.title('Train Labels Visualization')
sns.countplot(x=all_categories,palette='flare')
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
plt.suptitle('Train Images',fontsize=30)
path =image_path+'/'+'train/'
counter=0
for i,img in enumerate(os.listdir(path)):
    plt.subplot(4,4,i+1)
    full_img=plt.imread(path+img)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.xlabel(get_Label(all_categories[i]))
    plt.imshow(full_img,cmap=plt.cm.binary)
    if i==15:
        break
    

In [ ]:
df.category.value_counts()

In [ ]:
df.category=df.category.replace({1:'dog',0:'cat'})
df.head(10)

In [ ]:
df.category.value_counts()

In [ ]:
df['category'] = df['category'].replace({1:'dog',0:'cat'})
df.head(10)

# Data preperation


In [ ]:
x_train,x_val=train_test_split(df,random_state=42,shuffle=True,test_size=0.2)

In [ ]:
x_train.category.value_counts()

In [ ]:
x_val['category'].value_counts()

In [ ]:
x_train=x_train.reset_index(drop=True)
x_val=x_val.reset_index(drop=True)

In [ ]:
print('Train Images shape is        : ',x_train.shape)
print('Validation  Images  shape is : ',x_val.shape)

# Building CNN Model

In [ ]:
batch_size=64
image_gen=ImageDataGenerator( rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.5,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  fill_mode='nearest',
                                   horizontal_flip=True,
                                   rotation_range = 20, )



test_data_gen=ImageDataGenerator( rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.5,
                                  height_shift_range=0.2,
                                  width_shift_range=0.2,
                                  fill_mode='nearest',
                                   horizontal_flip=True,
                                   rotation_range = 20)



train=image_gen.flow_from_dataframe(x_train,
                                    image_path+'/'+'train/',
                                    shuffle=True,
                                    batch_size=batch_size,
                                    target_size=(128,128),
                                    x_col='image',
                                    y_col='category',
                                    class_mode='categorical'
                                    
                                   )

validate=test_data_gen.flow_from_dataframe(x_val,
                                           image_path+'/'+'train/',
                                           shuffle=True,
                                           x_col='image',
                                           y_col='category',
                                           target_size=(128,128),
                                           class_mode='categorical',
                                           batch_size=batch_size,
    
)

In [ ]:
for train_img,train_label in train:
    print('Image shape is : ',train_img.shape)
    print('Label Shape is :',train_label.shape)
    break

In [ ]:
for val_img,label_img in validate:
    print('Valdation images shape is : ',val_img.shape)
    print('Validate Label   shape is :' ,label_img.shape)
    break

In [ ]:
model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))



model.add(Dense(2,activation='softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="loss", mode="min",min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience = 20, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction]

In [ ]:
n_training_samples=len(train)
n_validate_samples=len(validate)
print(f'There are {n_training_samples} for training')
print(f'There are {n_validate_samples} for validating')

In [ ]:
history = model.fit(
    train,
    epochs=60,
    validation_data=validate,
    validation_steps=n_validate_samples//batch_size,
    steps_per_epoch =n_training_samples//batch_size,
    shuffle = True,
    callbacks=callbacks_list
    )

In [ ]:
history = model.fit(
    train,
    epochs=0,
    validation_data=validate,
    validation_steps=n_validate_samples//batch_size,
    steps_per_epoch =n_training_samples//batch_size,
    shuffle = True,
    callbacks=callbacks_list
    )

In [ ]:
#start bulding CNN Model
cnn_model = Sequential()
cnn_model = models.Sequential()
cnn_model.add(layers.Conv2D(64,(3,3),padding ='Same',activation = 'relu',input_shape=(128,128,3)))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(64,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2))
cnn_model.add(layers.Conv2D(128,(3,3),padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(layers.Conv2D(128,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(layers.Conv2D(256,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(BatchNormalization())
cnn_model.add(layers.Conv2D(256,(3,3) ,padding ='same',activation='relu'))
cnn_model.add(layers.MaxPooling2D(2,2)) 
cnn_model.add(BatchNormalization())
cnn_model.summary()
# cnn_model.add(Dropout(0.2))

In [ ]:
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(1024, activation='relu'))
cnn_model.add(BatchNormalization())
# cnn_model.add(Dropout(0.7))
cnn_model.add(layers.Dense(512, activation='relu'))
cnn_model.add(BatchNormalization())
# cnn_model.add(Dropout(0.3))
cnn_model.add(layers.Dense(2, activation ='softmax'))
cnn_model.summary()

In [ ]:
cnn_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="loss", mode="min",min_delta = 0,
                          patience = 10,
                          verbose = 1,
                          restore_best_weights = True)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)
callbacks_list = [ early, learning_rate_reduction]

In [ ]:
# Training model
n_training_samples = len(train)
n_validation_samples = len(validate)
history = cnn_model.fit(
    train,
    epochs=60,
    validation_data=validate,
    validation_steps=n_validation_samples//batch_size,
    # steps_per_epoch =n_training_samples//batch_size,
    shuffle = True,
    callbacks=callbacks_list
    )

In [ ]:
score, acc = cnn_model.evaluate(validate,batch_size=batch_size)                       
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
cnn_prediction = cnn_model.predict(validate)
cnn_prediction

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
#get image path
test_images = os.listdir(image_path+'/'+'test1')

#creat data frame
df_test =pd.DataFrame({
    'image': test_images,
})

df_test.head()

#prepare generator
val_data_gen  = ImageDataGenerator( rescale = 1./255, )  
        
testing = val_data_gen.flow_from_dataframe(
      df_test,
      image_path+'/'+'test1/',
      x_col='image',
      y_col= None,
      target_size=(128,128),
      class_mode=None,
      shuffle=True, 
      batch_size=batch_size
      )

In [ ]:
predection = cnn_model.predict(testing)
predection

In [ ]:
np.argmax(predection,axis=1)[0:100]

In [ ]:
plot_predection(model)